In [1]:
import warnings
import sys
sys.path.append('../../../../..')
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from models import *
from pytorch_lightning.loggers import TensorBoardLogger, CSVLogger
from pytorch_lightning.callbacks import LearningRateMonitor
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data_path = '../../../../../data/CIC_2018/no_preprocess/df_equal_Infilteration.parquet'
df = pd.read_parquet(data_path)
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,3389.0,6.0,1.519867e+09,3899579.0,14.0,7.0,1383.0,392.0,678.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
1,443.0,6.0,1.519808e+09,194.0,2.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
2,3389.0,6.0,1.519902e+09,15708594.0,4.0,2.0,43.0,0.0,43.0,0.0,...,20.0,451578.0,0.0,451578.0,451578.0,15300000.0,0.0,15300000.0,15300000.0,Benign
3,53.0,17.0,1.519896e+09,24712.0,2.0,2.0,74.0,106.0,37.0,37.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign
4,443.0,6.0,1.519792e+09,208.0,2.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Benign


In [3]:
df['Label'].isna().sum()

0

In [4]:
import numpy as np

infinite_values = df.isin([np.inf, -np.inf]).sum().sum()
print(f"Number of infinite values in the dataframe: {infinite_values}")

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

infinite_values = df.isin([np.inf, -np.inf]).sum().sum()
print(f"Number of infinite values in the dataframe: {infinite_values}")


Number of infinite values in the dataframe: 3944
Number of infinite values in the dataframe: 0


In [5]:
df['Label'].value_counts()

Label
Infilteration    160639
Benign           160308
Label                 0
Name: count, dtype: int64

In [6]:
df = df.drop(index=df.loc[df['Label'] == 'Label'].index)

In [7]:
df = df.dropna()

In [8]:
X = df.drop('Label', axis=1)
y = df['Label']
y = y.map({'Benign': 0, 'Infilteration': 1})
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, stratify=y)

In [9]:
df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [10]:
df_train.isna().sum().sum()

0

In [11]:
df_test.isna().sum().sum()

0

In [12]:
df_train['Label'].value_counts()

Label
1.0    128511
0.0    128246
Name: count, dtype: int64

In [13]:
df_train.info()
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 256757 entries, 182998 to 81882
Data columns (total 80 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Dst Port           256757 non-null  float32
 1   Protocol           256757 non-null  float16
 2   Timestamp          256757 non-null  float32
 3   Flow Duration      256757 non-null  float32
 4   Tot Fwd Pkts       256757 non-null  float32
 5   Tot Bwd Pkts       256757 non-null  float16
 6   TotLen Fwd Pkts    256757 non-null  float32
 7   TotLen Bwd Pkts    256757 non-null  float32
 8   Fwd Pkt Len Max    256757 non-null  float16
 9   Fwd Pkt Len Min    256757 non-null  float16
 10  Fwd Pkt Len Mean   256757 non-null  float16
 11  Fwd Pkt Len Std    256757 non-null  float16
 12  Bwd Pkt Len Max    256757 non-null  float16
 13  Bwd Pkt Len Min    256757 non-null  float16
 14  Bwd Pkt Len Mean   256757 non-null  float16
 15  Bwd Pkt Len Std    256757 non-null  float16
 16  Flo

In [14]:
df_test['Label'].value_counts()

Label
1.0    32128
0.0    32062
Name: count, dtype: int64

In [15]:
datamodule = ExpertPretrainDataModule(df_train, df_test, binarize_on_label=1)

In [16]:
expert_model = ExpertModel(79, [256, 128])
print(expert_model)

ExpertModel(
  (model): Sequential(
    (0): BatchNorm1d(79, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=79, out_features=256, bias=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)


In [17]:
logger = TensorBoardLogger(f"../../../../../logs/expert/no_preprocess/autoencoder/infiltration", name="expert_tensorboard")
csv_logger = CSVLogger(f"../../../../../logs/expert/no_preprocess/autoencoder/infiltration", name="expert_csv")
lr_monitor = LearningRateMonitor(logging_interval='epoch')


trainer = pl.Trainer(
        max_epochs=50,
        logger=[logger, csv_logger],
        callbacks=[lr_monitor],
        accelerator='gpu',
        precision='64-true',
)

trainer.fit(expert_model, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
2024-10-06 20:52:26.827989: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-06 20:52:26.840803: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-06 20:52:26.846177: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=50` reached.
